In [50]:
import pandas as pd
import numpy as np

In [46]:
df =pd.DataFrame([[4,9]]*3,columns = ['A','B'])

In [47]:
df

,A,B
0,4,9
1,4,9
2,4,9


In [64]:
df.apply(np.mean, axis=1)

0    6.5
1    6.5
2    6.5
dtype: float64

In [63]:
df.apply(np.sqrt)

,A,B
0,2.0,3.0
1,2.0,3.0
2,2.0,3.0


In [66]:
df.apply(lambda x: x.A + x.B, axis=1)

0    13
1    13
2    13
dtype: int64

In [7]:
def Mul_a(x):
    return x.A * x.B
df['C'] = df.apply(Mul_a, axis=1)

In [41]:
housing = pd.DataFrame({'城市':['北京','上海','杭州','深圳'],'房价':[20,15,5,10]})

In [42]:
def is_yixian(x):
    if x['城市'] in ['北京','上海','深圳']:
        return True
    return False

In [43]:
housing['一线'] = housing.apply(is_yixian, axis = 1)

In [44]:
housing 

,城市,房价,一线
0,北京,20,True
1,上海,15,True
2,杭州,5,False
3,深圳,10,True


In [37]:
score_table = pd.DataFrame(dict(name = ['君豪','哲宇','承钰','承熙','翰辰','少杰','天泽','毅康','俊杰','天毅','德华'], score = [23,54,'',13,24,57,'',16,27,86,16]))
staff_table = pd.DataFrame(dict(name = ['君豪','天毅','德华','哲宇','承钰','承熙','翰辰','少杰','天泽','毅康','俊杰'], level = [3,4,5,3,4,3,4,3,4,5,3], project = [10,6,15,7,8,9,3,17,20,9,12],working_hours = [45,34,57,46,35,55,57,36,40,43,39], code = [100,200,360,360,530,478,123,245,543,643,275]))

In [20]:
score_table

,name,score
0,君豪,23
1,哲宇,54
2,承钰,
3,承熙,13
4,翰辰,24
5,少杰,57
6,天泽,
7,毅康,16
8,俊杰,27
9,天毅,86


In [21]:
staff_table

,name,level,project,working_hours,code
0,君豪,3,10,45,100
1,天毅,4,6,34,200
2,德华,5,15,57,360
3,哲宇,3,7,46,360
4,承钰,4,8,35,530
5,承熙,3,9,55,478
6,翰辰,4,3,57,123
7,少杰,3,17,36,245
8,天泽,4,20,40,543
9,毅康,5,9,43,643


  3级 加班时间 * 0.1 + 完成项目数 * 0.5 + 有效代码行数 * 0.1，  
  4级及以上  加班时长 * 0.08 + 完成项目数 * 0.8 + 有效代码行数 * 0.03  
  加班时长 =  工作时长 - 40 且加班时长 ≥ 0  
  先需要将数据填补到原有的工作表中(因原表中的评分更加科学，填补数据是无奈之举，所以如果有评分的员工，以原有评分为准)

In [32]:
def Score(x):
    if not x.score:
        if x.level == 3:
            return x.working_hours * 0.1 + x.project * 0.5 + x.code * 0.1
        else:
            if x.working_hours - 40 > 0:
                return (x.working_hours - 40) * 0.08 + x.project * 0.8 + x.code * 0.03
            else: 
                return x.project * 0.8 + x.code * 0.03
    else:
        return x.score

In [33]:
new_t = score_table.merge(staff_table, on = 'name')

In [35]:
new_t['score'] = new_t.apply(Score, axis = 1)

In [38]:
new_t

,name,score,level,project,working_hours,code
0,君豪,23.00,3,10,45,100
1,哲宇,54.00,3,7,46,360
2,承钰,22.30,4,8,35,530
3,承熙,13.00,3,9,55,478
4,翰辰,24.00,4,3,57,123
5,少杰,57.00,3,17,36,245
6,天泽,32.29,4,20,40,543
7,毅康,16.00,5,9,43,643
8,俊杰,27.00,3,12,39,275
9,天毅,86.00,4,6,34,200


In [39]:
def calc_score(staff):
    if staff.level == 3:
        score = max(staff.overtime_hours,0) * 0.1 + staff.project * 0.5 + staff.code * 0.1
    else:
        score = max(staff.overtime_hours,0) * 0.08 + staff.project * 0.8 + staff.code * 0.03
    return score
    
staff_table.set_index(staff_table.name,drop=True,inplace=True)
staff_table = staff_table.reindex(score_table.name).reset_index(drop=True)

staff_table['overtime_hours'] = staff_table.working_hours - 40
staff_table['score'] = staff_table.apply(lambda x:calc_score(x),axis = 1)
score_table.score.loc[score_table.score == ''] = staff_table.score

In [40]:
score_table

,name,score
0,君豪,23
1,哲宇,54
2,承钰,22.3
3,承熙,13
4,翰辰,24
5,少杰,57
6,天泽,32.29
7,毅康,16
8,俊杰,27
9,天毅,86
